In [2]:
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,ShuffleSplit
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  confusion_matrix
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier


In [6]:
LabelY='Label'
errmsg='USAGE: learn.py DesiredModel C'
models_list=['linear_svm','tree','logistic','forest']
argl=len(sys.argv)
if(argl>3):
    print(errmsg)
    exit(1)

desired_model='forest'
C=3.0


if(argl>=2 and sys.argv[1] in models_list):
    desired_model=sys.argv[1]

if(argl==3):
    C=float(sys.argv[3])
    



In [38]:
#Reading into pandas dataframe from excel file.

#df_normal=pd.read_excel('C:/Users/dell/Desktop/SCPS/df_normal.xlsx',skipinitialspace='true')
#df_attack=pd.read_excel(r"C:\Users\dell\Desktop\SCPS\ListAttacks.xlsx",skipinitialspace='true')

# Read the Excel file
df_attack = pd.read_excel(r"C:\Users\dell\Desktop\SCPS\ListAttacks.xlsx")
df_normal = pd.read_excel(r"C:\Users\dell\Desktop\SCPS\df_normal.xlsx")



In [24]:
df_normal.head()

,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Label
0,2.445953,630.1245,2,1,1,267.1110,8.419188,321.7380,0.000000,1,...,2,1,255.3672,1.185403,194.1329,0.000064,1,1,1,Normal
1,0.000000,527.6356,1,2,1,266.2138,8.349013,336.8880,2.455900,2,...,2,1,254.8705,1.409669,193.8125,0.000128,1,1,1,Normal
2,2.461006,512.6410,2,2,1,266.4701,8.378492,329.4283,2.445007,2,...,2,1,251.2977,1.041233,190.2557,0.000000,1,1,1,Normal
3,2.614419,522.3757,2,2,1,258.9720,8.350295,334.2476,2.451158,2,...,2,1,255.8639,1.041233,194.5014,0.000000,1,1,1,Normal
4,0.000000,655.2463,1,2,1,260.3820,8.377851,332.8890,2.457053,2,...,2,1,250.0641,1.185403,188.8779,0.000064,1,1,1,Normal


In [25]:
df_attack.head()

,Attack #,Start Time,End Time,Attack Point,Start State,Attack,Actual Change,Expected Impact or attacker intent,Unexpected Outcome
0,1.0,28/12/2015 10:29:14,10:44:53,MV‐101,MV‐101 is closed,Open MV‐101,Yes,Tank overflow,NaN
1,2.0,28/12/2015 10:51:08,10:58:30,P‐102,P‐101 is on where as P‐102 is off,Turn on P‐102,Yes,Pipe bursts,NaN
2,3.0,28/12/2015 11:22:00,11:28:22,LIT‐101,Water level between L and H,Increase by 1 mm every second,No,Tank Underflow; Damage P‐101,NaN
3,4.0,28/12/2015 11:47:39,11:54:08,MV‐504,MV‐504 is closed,Open MV‐504,Yes,Halt RO shut down sequence; Reduce life of RO,No impact
4,NaN,28/12/2015 11:58:20,NaN,No Physical Impact Attack,NaN,NaN,NaN,NaN,NaN


In [26]:
# Strip whitespace from the headers and all string-type cells
df_attack.columns = df_attack.columns.str.strip()  # Strip whitespace from headers
df_attack = df_attack.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Strip whitespace from string cells



C:\Users\dell\AppData\Local\Temp\ipykernel_14112\1175859441.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_attack = df_attack.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Strip whitespace from string cells


In [27]:
# Now df_attack is clean and ready for further processing
df_attack.head()


,Attack #,Start Time,End Time,Attack Point,Start State,Attack,Actual Change,Expected Impact or attacker intent,Unexpected Outcome
0,1.0,28/12/2015 10:29:14,10:44:53,MV‐101,MV‐101 is closed,Open MV‐101,Yes,Tank overflow,NaN
1,2.0,28/12/2015 10:51:08,10:58:30,P‐102,P‐101 is on where as P‐102 is off,Turn on P‐102,Yes,Pipe bursts,NaN
2,3.0,28/12/2015 11:22:00,11:28:22,LIT‐101,Water level between L and H,Increase by 1 mm every second,No,Tank Underflow; Damage P‐101,NaN
3,4.0,28/12/2015 11:47:39,11:54:08,MV‐504,MV‐504 is closed,Open MV‐504,Yes,Halt RO shut down sequence; Reduce life of RO,No impact
4,NaN,28/12/2015 11:58:20,NaN,No Physical Impact Attack,NaN,NaN,NaN,NaN,NaN


In [28]:
# Strip whitespace from the headers and all string-type cells
df_normal.columns = df_normal.columns.str.strip()  # Strip whitespace from headers
df_normal = df_attack.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Strip whitespace from string cells


C:\Users\dell\AppData\Local\Temp\ipykernel_14112\198337483.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_normal = df_attack.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Strip whitespace from string cells


In [29]:
df_normal.head()

,Attack #,Start Time,End Time,Attack Point,Start State,Attack,Actual Change,Expected Impact or attacker intent,Unexpected Outcome
0,1.0,28/12/2015 10:29:14,10:44:53,MV‐101,MV‐101 is closed,Open MV‐101,Yes,Tank overflow,NaN
1,2.0,28/12/2015 10:51:08,10:58:30,P‐102,P‐101 is on where as P‐102 is off,Turn on P‐102,Yes,Pipe bursts,NaN
2,3.0,28/12/2015 11:22:00,11:28:22,LIT‐101,Water level between L and H,Increase by 1 mm every second,No,Tank Underflow; Damage P‐101,NaN
3,4.0,28/12/2015 11:47:39,11:54:08,MV‐504,MV‐504 is closed,Open MV‐504,Yes,Halt RO shut down sequence; Reduce life of RO,No impact
4,NaN,28/12/2015 11:58:20,NaN,No Physical Impact Attack,NaN,NaN,NaN,NaN,NaN


In [30]:
df_normal=df_normal.drop_duplicates()
df_attack=df_attack.drop_duplicates()


In [33]:
#Strip blankspaces from data
df_normal=df_normal.apply(lambda x: x.str.strip() if x.dtype == "object" else x).rename(columns=lambda x: x.strip())
df_attack=df_attack.apply(lambda x: x.str.strip() if x.dtype == "object" else x).rename(columns=lambda x: x.strip())



In [34]:
df_normal.head()

,Attack #,Start Time,End Time,Attack Point,Start State,Attack,Actual Change,Expected Impact or attacker intent,Unexpected Outcome
0,1.0,28/12/2015 10:29:14,10:44:53,MV‐101,MV‐101 is closed,Open MV‐101,Yes,Tank overflow,NaN
1,2.0,28/12/2015 10:51:08,10:58:30,P‐102,P‐101 is on where as P‐102 is off,Turn on P‐102,Yes,Pipe bursts,NaN
2,3.0,28/12/2015 11:22:00,11:28:22,LIT‐101,Water level between L and H,Increase by 1 mm every second,No,Tank Underflow; Damage P‐101,NaN
3,4.0,28/12/2015 11:47:39,11:54:08,MV‐504,MV‐504 is closed,Open MV‐504,Yes,Halt RO shut down sequence; Reduce life of RO,No impact
4,NaN,28/12/2015 11:58:20,NaN,No Physical Impact Attack,NaN,NaN,NaN,NaN,NaN


In [41]:
# Check column names and data types
print("df_attack columns and types:\n", df_attack.dtypes)
print("df_normal columns and types:\n", df_normal.dtypes)



df_attack columns and types:
 Attack #                               float64
Start Time                              object
End Time                                object
Attack Point                            object
Start State                             object
Attack                                  object
Actual Change                           object
Expected Impact or attacker intent      object
Unexpected Outcome                      object
dtype: object
df_normal columns and types:
 FIT101     float64
LIT101     float64
MV101        int64
P101         int64
P102         int64
AIT201     float64
AIT202     float64
AIT203     float64
FIT201     float64
MV201        int64
P201         int64
P202         int64
P203         int64
P204         int64
P205         int64
P206         int64
DPIT301    float64
FIT301     float64
LIT301     float64
MV301        int64
MV302        int64
MV303        int64
MV304        int64
P301         int64
P302         int64
AIT401     float64
AIT402   

In [42]:
# Concatenate with `join='outer'` and `ignore_index=True`
df = pd.concat([df_attack, df_normal], join='outer', ignore_index=True)


In [43]:
df.head()

,Attack #,Start Time,End Time,Attack Point,Start State,Attack,Actual Change,Expected Impact or attacker intent,Unexpected Outcome,FIT101,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Label
0,1.0,28/12/2015 10:29:14,10:44:53,MV‐101,MV‐101 is closed,Open MV‐101,Yes,Tank overflow,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,28/12/2015 10:51:08,10:58:30,P‐102,P‐101 is on where as P‐102 is off,Turn on P‐102,Yes,Pipe bursts,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,28/12/2015 11:22:00,11:28:22,LIT‐101,Water level between L and H,Increase by 1 mm every second,No,Tank Underflow; Damage P‐101,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,28/12/2015 11:47:39,11:54:08,MV‐504,MV‐504 is closed,Open MV‐504,Yes,Halt RO shut down sequence; Reduce life of RO,No impact,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,28/12/2015 11:58:20,NaN,No Physical Impact Attack,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
sz,n_feat=df.shape
print("Dataset Read, Size=",sz)
print('Number of features=',n_feat-1)


Dataset Read, Size= 20041
Number of features= 60


In [45]:
df_temp=df


In [46]:
#print("Column names:", df.columns)
df_normal.head()

,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Label
0,2.445953,630.1245,2,1,1,267.1110,8.419188,321.7380,0.000000,1,...,2,1,255.3672,1.185403,194.1329,0.000064,1,1,1,Normal
1,0.000000,527.6356,1,2,1,266.2138,8.349013,336.8880,2.455900,2,...,2,1,254.8705,1.409669,193.8125,0.000128,1,1,1,Normal
2,2.461006,512.6410,2,2,1,266.4701,8.378492,329.4283,2.445007,2,...,2,1,251.2977,1.041233,190.2557,0.000000,1,1,1,Normal
3,2.614419,522.3757,2,2,1,258.9720,8.350295,334.2476,2.451158,2,...,2,1,255.8639,1.041233,194.5014,0.000000,1,1,1,Normal
4,0.000000,655.2463,1,2,1,260.3820,8.377851,332.8890,2.457053,2,...,2,1,250.0641,1.185403,188.8779,0.000064,1,1,1,Normal


In [47]:
#Encoding Label Column to 0,1 for different classes. 
le=LabelEncoder()
#LabelY='Label'
#LabelY = df_normal.iloc[:,-1]

df[LabelY]=le.fit_transform(df[LabelY])


In [48]:



X=df.drop(LabelY,axis=1)
y=df[LabelY]


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=40,stratify=y)


In [50]:
df_train=pd.DataFrame(columns=df.columns,data=np.column_stack((X_train,y_train)))
df_test=pd.DataFrame(columns=df.columns,data=np.column_stack((X_test,y_test)))


In [51]:

df_train.to_excel('df_train_multiclass.xlsx',index=False)
df_test.to_excel('df_test_multiclass.xlsx',index=False)


In [62]:
# Convert Start Time and End Time columns to Unix timestamp format
df['Start Time'] = pd.to_datetime(df['Start Time'], errors='coerce').astype(int) / 10**9
df['End Time'] = pd.to_datetime(df['End Time'], errors='coerce').astype(int) / 10**9


C:\Users\dell\AppData\Local\Temp\ipykernel_14112\1515156524.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Start Time'] = pd.to_datetime(df['Start Time'], errors='coerce').astype(int) / 10**9
C:\Users\dell\AppData\Local\Temp\ipykernel_14112\1515156524.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['End Time'] = pd.to_datetime(df['End Time'], errors='coerce').astype(int) / 10**9


In [63]:
# Replace 'Yes' with 1 and 'No' with 0 in the Attack column
df['Attack'] = df['Attack'].map({'Yes': 1, 'No': 0})


In [65]:
df.columns = df.columns.str.strip()  # Remove any leading or trailing spaces


In [66]:
print(df.columns)

Index(['Attack #', 'Start Time', 'End Time', 'Attack Point', 'Start State',
       'Attack', 'Actual Change', 'Expected Impact or attacker intent',
       'Unexpected Outcome', 'FIT101', 'LIT101', 'MV101', 'P101', 'P102',
       'AIT201', 'AIT202', 'AIT203', 'FIT201', 'MV201', 'P201', 'P202', 'P203',
       'P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302',
       'MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402', 'FIT401',
       'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501', 'AIT502',
       'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501',
       'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P601', 'P602', 'P603',
       'Label'],
      dtype='object')


In [67]:
# Use one-hot encoding for all other categorical columns
df = pd.get_dummies(df, columns=['Attack Point', 'Start State', 'Actual Change', 
                                 'Expected Impact or attacker intent', 'Unexpected Outcome'])


In [68]:
# Identify and display any remaining non-numeric columns
non_numeric_columns = df.select_dtypes(include=['object']).columns
print("Non-numeric columns:", non_numeric_columns)

# Convert any remaining non-numeric columns if needed or investigate specific cells


Non-numeric columns: Index([], dtype='object')


In [80]:

#Centering data by mean i.e. subtracting mean from each data
#scaler=StandardScaler()



In [81]:
X_train

,Attack #,Start Time,End Time,Attack Point,Start State,Attack,Actual Change,Expected Impact or attacker intent,Unexpected Outcome,FIT101,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
13715,NaN,-9.223372e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.481504,...,0.308298,2.0,1.0,254.1015,1.169384,192.9794,0.000064,1.0,1.0,1.0
12890,NaN,-9.223372e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.660859,...,0.310284,2.0,1.0,255.6876,1.105309,194.2611,0.000064,1.0,1.0,1.0
12860,NaN,-9.223372e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.479903,...,0.310284,2.0,1.0,254.8705,0.977157,193.8926,0.000064,1.0,1.0,1.0
2827,NaN,-9.223372e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.617942,...,0.308426,2.0,1.0,251.6342,1.105309,190.3839,0.000064,1.0,1.0,1.0
12106,NaN,-9.223372e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.609935,...,0.308298,2.0,1.0,251.7624,1.041233,190.4800,0.000000,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2748,NaN,-9.223372e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.618582,...,0.308362,2.0,1.0,256.3125,1.057252,194.8539,0.000000,1.0,1.0,1.0
4689,NaN,-9.223372e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.665983,...,0.310348,2.0,1.0,255.4473,1.441707,194.5655,0.000128,1.0,1.0,1.0
10315,NaN,-9.223372e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.043237,...,0.304391,2.0,1.0,247.2122,1.169384,186.8752,0.000000,1.0,1.0,1.0
5330,NaN,-9.223372e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,...,0.306761,2.0,1.0,246.8758,1.041233,186.2343,0.000000,1.0,1.0,1.0


In [82]:
# Filter out non-numeric columns in X_train
X_train_numeric = X_train.select_dtypes(include=[np.number])
X_test_numeric = X_test.select_dtypes(include=[np.number])


In [83]:
# Initialize the scaler
scaler = StandardScaler()

# Scale the numeric data
X_train_scaled = scaler.fit_transform(X_train_numeric)
X_test_scaled = scaler.transform(X_test_numeric)


In [84]:
models_list={'linear_svm':SVC(kernel='linear',C=C,class_weight='balanced'),
             'tree':DecisionTreeClassifier(class_weight='balanced'),
             'logistic':LogisticRegression(class_weight='balanced',C=C),
             'forest':RandomForestClassifier(class_weight='balanced'),   
             'BNB':BernoulliNB(),
             'Extree':ExtraTreeClassifier(),
             'Extrees':ExtraTreesClassifier(),
             'GNB':GaussianNB(),
             'KNN':KNeighborsClassifier(),
             'LDA':LinearDiscriminantAnalysis(),
             'LinearSVC':LinearSVC(multi_class='crammer_singer'),
             'logisticCV':LogisticRegressionCV(multi_class='multinomial'),
             'MLP':MLPClassifier(),
             'NearestCentroid':NearestCentroid(),
             'QDA':QuadraticDiscriminantAnalysis(),
             'RNC':RadiusNeighborsClassifier(),
             'GPC':GaussianProcessClassifier(multi_class = 'one_vs_rest')}



In [1]:
'''
#5 different splits of 25%,50% and taking average on validation score with some stdev
#This way we can effectively use all 75%  for training.(Leave One Out CV Method below)
cv = ShuffleSplit(n_splits=5, test_size=int(0.25*sz), random_state=40)
scores=cross_val_score(clf,X_train,y_train,cv=cv)
print('scores', scores)
print("Validation Accuracy=%.4f Std(+/- %0.4f)" % (scores.mean()*100,scores.std()*200))

y_pred=clf.predict(X_test)


conf_mat=confusion_matrix(y_test,y_pred)
print("\n\nConfusion Matrix:-\n",list(le.inverse_transform([0,1,2,3,4,5,6])),'\n',conf_mat)
print()
print("Testing Accuracy= ", clf.score(X_test,y_test)*100)

'''

'\n#5 different splits of 25%,50% and taking average on validation score with some stdev\n#This way we can effectively use all 75%  for training.(Leave One Out CV Method below)\ncv = ShuffleSplit(n_splits=5, test_size=int(0.25*sz), random_state=40)\nscores=cross_val_score(clf,X_train,y_train,cv=cv)\nprint(\'scores\', scores)\nprint("Validation Accuracy=%.4f Std(+/- %0.4f)" % (scores.mean()*100,scores.std()*200))\n\ny_pred=clf.predict(X_test)\n\n\nconf_mat=confusion_matrix(y_test,y_pred)\nprint("\n\nConfusion Matrix:-\n",list(le.inverse_transform([0,1,2,3,4,5,6])),\'\n\',conf_mat)\nprint()\nprint("Testing Accuracy= ", clf.score(X_test,y_test)*100)\n\n'

In [86]:
clf=models_list[desired_model]


In [87]:
print('\n\nTraining ',desired_model,' model...')





Training  forest  model...


In [2]:
# Convert datetime columns to numeric, if needed
# Assuming 'Start Time' and 'End Time' columns exist and contain datetime data
if 'Start Time' in X_train.columns and 'End Time' in X_train.columns:
    X_train['Start Time'] = pd.to_datetime(X_train['Start Time'], errors='coerce').astype(int) / 10**9
    X_train['End Time'] = pd.to_datetime(X_train['End Time'], errors='coerce').astype(int) / 10**9
    X_test['Start Time'] = pd.to_datetime(X_test['Start Time'], errors='coerce').astype(int) / 10**9
    X_test['End Time'] = pd.to_datetime(X_test['End Time'], errors='coerce').astype(int) / 10**9

# Encode remaining categorical columns
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Align X_train and X_test columns
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)


"# Convert datetime columns to numeric, if needed\n# Assuming 'Start Time' and 'End Time' columns exist and contain datetime data\nif 'Start Time' in X_train.columns and 'End Time' in X_train.columns:\n    X_train['Start Time'] = pd.to_datetime(X_train['Start Time'], errors='coerce').astype(int) / 10**9\n    X_train['End Time'] = pd.to_datetime(X_train['End Time'], errors='coerce').astype(int) / 10**9\n    X_test['Start Time'] = pd.to_datetime(X_test['Start Time'], errors='coerce').astype(int) / 10**9\n    X_test['End Time'] = pd.to_datetime(X_test['End Time'], errors='coerce').astype(int) / 10**9\n\n# Encode remaining categorical columns\nX_train = pd.get_dummies(X_train, drop_first=True)\nX_test = pd.get_dummies(X_test, drop_first=True)\n\n# Align X_train and X_test columns\nX_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)\n"

In [90]:
clf.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced')

In [93]:
#5 different splits of 25%,50% and taking average on validation score with some stdev
#This way we can effectively use all 75%  for training.(Leave One Out CV Method below)
cv = ShuffleSplit(n_splits=5, test_size=int(0.25*sz), random_state=40)
scores=cross_val_score(clf,X_train,y_train,cv=cv)
print('scores', scores)
print("Validation Accuracy=%.4f Std(+/- %0.4f)" % (scores.mean()*100,scores.std()*200))


scores [1. 1. 1. 1. 1.]
Validation Accuracy=100.0000 Std(+/- 0.0000)


In [96]:
# Fit the label encoder on combined labels from both train and test sets
le = LabelEncoder()
le.fit(np.concatenate((y_train, y_test)))


LabelEncoder()

In [97]:
y_train = le.transform(y_train)
y_test = le.transform(y_test)


In [100]:
y_pred = clf.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)

# Print the labels and the confusion matrix
print("\n\nConfusion Matrix:-\n", list(le.inverse_transform(range(len(le.classes_)))), '\n', conf_mat)
print("\nTesting Accuracy= ", clf.score(X_test, y_test) * 100)







Confusion Matrix:-
 [np.int64(0), np.int64(1)] 
 [[5001    0]
 [   0   10]]

Testing Accuracy=  100.0


In [104]:
print("Classes in LabelEncoder:", le.classes_)


Classes in LabelEncoder: [0 1]


In [105]:
# Define class names and testing accuracy
class_names = list(le.inverse_transform([0, 1]))
accuracy = clf.score(X_test, y_test) * 100

# Print the confusion matrix and testing accuracy
print("\nConfusion Matrix:\n", class_names, '\n', conf_mat)
print("\nTesting Accuracy = {:.2f}%".format(accuracy))
conf_mat = confusion_matrix(y_test, y_pred)

'''conf_mat=confusion_matrix(y_test,y_pred)
print("\n\nConfusion Matrix:-\n",list(le.inverse_transform([0,1,2,3,4,5,6])),'\n',conf_mat)
print()
print("Testing Accuracy= ", clf.score(X_test,y_test)*100)
'''


Confusion Matrix:
 [np.int64(0), np.int64(1)] 
 [[5001    0]
 [   0   10]]

Testing Accuracy = 100.00%


'conf_mat=confusion_matrix(y_test,y_pred)\nprint("\n\nConfusion Matrix:-\n",list(le.inverse_transform([0,1,2,3,4,5,6])),\'\n\',conf_mat)\nprint()\nprint("Testing Accuracy= ", clf.score(X_test,y_test)*100)\n'